In [ ]:
# 🧠 Classify Tasks: ACTION vs RESULT

This Codelab demonstrates how to build a transformer-based classifier to distinguish between simple one-step tasks (`ACTION`)
and more complex multi-step tasks (`RESULT`) using Hugging Face's Transformers library.

Tested with A100 runtime

In [13]:
# Install all necessary libraries.
!pip install -q transformers datasets scikit-learn

In [14]:
# Import needed dependecies

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.model_selection import train_test_split
from datasets import Dataset

import pandas as pd
import torch

In [15]:
# Define training dataset

data = [
    # ACTIONS
    {"text": "Send an email to the manager", "label": "ACTION"},
    {"text": "Book a table for two at the restaurant", "label": "ACTION"},
    {"text": "Take out the trash", "label": "ACTION"},
    {"text": "Pay the electricity bill", "label": "ACTION"},
    {"text": "Call mom", "label": "ACTION"},
    {"text": "Refill the printer with paper", "label": "ACTION"},
    {"text": "Schedule a meeting with the team", "label": "ACTION"},
    {"text": "Water the plants", "label": "ACTION"},
    {"text": "Pick up laundry", "label": "ACTION"},
    {"text": "Post the letter", "label": "ACTION"},
    {"text": "Make a cup of tea", "label": "ACTION"},
    {"text": "Send the project report", "label": "ACTION"},
    {"text": "Buy dog food", "label": "ACTION"},
    {"text": "Install the app on your phone", "label": "ACTION"},
    {"text": "Reply to the client message", "label": "ACTION"},
    {"text": "Take the medicine", "label": "ACTION"},
    {"text": "Turn off the lights", "label": "ACTION"},
    {"text": "Print the invoice", "label": "ACTION"},
    {"text": "Charge your laptop", "label": "ACTION"},
    {"text": "Update the software", "label": "ACTION"},
    {"text": "Cancel the gym subscription", "label": "ACTION"},
    {"text": "Sign the document", "label": "ACTION"},
    {"text": "Submit the form", "label": "ACTION"},
    {"text": "Write a thank-you note", "label": "ACTION"},
    {"text": "Schedule an appointment with the doctor", "label": "ACTION"},
    {"text": "Share the meeting link", "label": "ACTION"},
    {"text": "Download the presentation slides", "label": "ACTION"},
    {"text": "Change your email password", "label": "ACTION"},
    {"text": "Backup your photos", "label": "ACTION"},
    {"text": "Print boarding passes", "label": "ACTION"},
    {"text": "Clean the whiteboard", "label": "ACTION"},
    {"text": "Replace the lightbulb", "label": "ACTION"},
    {"text": "Approve the invoice", "label": "ACTION"},
    {"text": "Send flowers to Alice", "label": "ACTION"},
    {"text": "Copy the files to USB", "label": "ACTION"},
    {"text": "Record a short video", "label": "ACTION"},
    {"text": "Submit the ticket to IT", "label": "ACTION"},
    {"text": "Book a taxi to the airport", "label": "ACTION"},
    {"text": "Mark the task as complete", "label": "ACTION"},
    {"text": "Pay for the parking", "label": "ACTION"},
    {"text": "Log out from all devices", "label": "ACTION"},
    {"text": "Post an update on LinkedIn", "label": "ACTION"},
    {"text": "Mute notifications for 1 hour", "label": "ACTION"},
    {"text": "Pick up coffee on the way", "label": "ACTION"},
    {"text": "Scan the QR code", "label": "ACTION"},
    {"text": "Attach the invoice to the email", "label": "ACTION"},
    {"text": "Text John your ETA", "label": "ACTION"},
    {"text": "Forward the contract", "label": "ACTION"},
    {"text": "Reset your Wi-Fi router", "label": "ACTION"},
    {"text": "Switch to presentation mode", "label": "ACTION"},
    {"text": "Turn on the air conditioner", "label": "ACTION"},
    {"text": "Book an appointment at the car repair shop", "label": "ACTION"},
    {"text": "Take the car to the mechanic", "label": "ACTION"},
    {"text": "Call the garage to ask about the repair", "label": "ACTION"},
    {"text": "Drop off the car for service", "label": "ACTION"},
    {"text": "Clean the kitchen. It takes about 30 minutes", "label": "ACTION"},
    {"text": "Write a short summary of the meeting. Shouldn't take more than an hour", "label": "ACTION"},
    {"text": "Go grocery shopping for the week. Should take around 90 minutes", "label": "ACTION"},
    {"text": "Fix the broken chair. Should take less than an hour", "label": "ACTION"},
    {"text": "Respond to all emails from this morning. Should take about 45 minutes", "label": "ACTION"},
    # RESULTS
    {"text": "Organize a wedding", "label": "RESULT"},
    {"text": "Launch a new product", "label": "RESULT"},
    {"text": "Prepare a marketing strategy", "label": "RESULT"},
    {"text": "Renovate the kitchen", "label": "RESULT"},
    {"text": "Plan a family vacation", "label": "RESULT"},
    {"text": "Design a new website", "label": "RESULT"},
    {"text": "Write a novel", "label": "RESULT"},
    {"text": "Develop a mobile app", "label": "RESULT"},
    {"text": "Create a business plan", "label": "RESULT"},
    {"text": "Move to a new apartment", "label": "RESULT"},
    {"text": "Organize the annual company event", "label": "RESULT"},
    {"text": "Conduct user research for a new feature", "label": "RESULT"},
    {"text": "Build a personal brand on social media", "label": "RESULT"},
    {"text": "Implement a new CRM system", "label": "RESULT"},
    {"text": "Recruit a new engineering team", "label": "RESULT"},
    {"text": "Coordinate a fundraising campaign", "label": "RESULT"},
    {"text": "Improve employee onboarding process", "label": "RESULT"},
    {"text": "Start an online business", "label": "RESULT"},
    {"text": "Write and defend a thesis", "label": "RESULT"},
    {"text": "Produce a podcast series", "label": "RESULT"},
    {"text": "Organize the office relocation", "label": "RESULT"},
    {"text": "Plan a product launch event", "label": "RESULT"},
    {"text": "Set up an automated sales funnel", "label": "RESULT"},
    {"text": "Develop a course for online learning", "label": "RESULT"},
    {"text": "Migrate company data to the cloud", "label": "RESULT"},
    {"text": "Train new customer support staff", "label": "RESULT"},
    {"text": "Redesign the employee handbook", "label": "RESULT"},
    {"text": "Publish a weekly company newsletter", "label": "RESULT"},
    {"text": "Integrate payment system into the platform", "label": "RESULT"},
    {"text": "Optimize warehouse logistics", "label": "RESULT"},
    {"text": "Transition the team to agile workflows", "label": "RESULT"},
    {"text": "Build a knowledge base for users", "label": "RESULT"},
    {"text": "Create a full-scale financial forecast", "label": "RESULT"},
    {"text": "Launch a customer loyalty program", "label": "RESULT"},
    {"text": "Audit the cybersecurity infrastructure", "label": "RESULT"},
    {"text": "Implement employee performance reviews", "label": "RESULT"},
    {"text": "Prepare annual sustainability report", "label": "RESULT"},
    {"text": "Overhaul internal documentation", "label": "RESULT"},
    {"text": "Create a pitch deck for investors", "label": "RESULT"},
    {"text": "Conduct a competitive market analysis", "label": "RESULT"},
    {"text": "Update all legal contracts with partners", "label": "RESULT"},
    {"text": "Build an onboarding flow for users", "label": "RESULT"},
    {"text": "Coordinate volunteer activities for the team", "label": "RESULT"},
    {"text": "Evaluate different recruitment platforms", "label": "RESULT"},
    {"text": "Roll out a remote work policy", "label": "RESULT"},
    {"text": "Prepare and deliver a webinar", "label": "RESULT"},
    {"text": "Set up the data analytics pipeline", "label": "RESULT"},
    {"text": "Launch a blog and editorial calendar", "label": "RESULT"},
    {"text": "Lead the rebranding campaign", "label": "RESULT"},
    {"text": "Design and implement customer surveys", "label": "RESULT"},
    {"text": "Revamp the internal feedback system", "label": "RESULT"},
    {"text": "Fix the washing machine. I don't know how to do it", "label": "RESULT"},
    {"text": "Paint the house. I'll need to hire someone", "label": "RESULT"},
    {"text": "Repair the air conditioner. It's too technical for me", "label": "RESULT"},
    {"text": "Replace the brake pads. I can't do it myself", "label": "RESULT"},
    {"text": "Clean the whole house. It usually takes 4 to 5 hours", "label": "RESULT"},
    {"text": "Write the annual financial report. It takes more than a day", "label": "RESULT"},
    {"text": "Prepare materials and slides for the client presentation. It will take several hours", "label": "RESULT"},
    {"text": "Organize and label the entire photo archive. Could take a full afternoon", "label": "RESULT"},
    {"text": "Update and redesign the project documentation. Requires several hours", "label": "RESULT"},
    {"text": "I have all the tools. I can fix the car in 10 minutes", "label": "ACTION"},
    {"text": "The problem is simple. I can fix the car right away", "label": "ACTION"},
    {"text": "I already have the part. Replacing it takes just 5 minutes", "label": "ACTION"},
    {"text": "Changing the tire will take me 15 minutes max", "label": "ACTION"},
    {"text": "Fixing the mirror is quick. I’ve done it before", "label": "ACTION"},
]

# Map labels to integers
label2id = {"ACTION": 0, "RESULT": 1}
id2label = {0: "ACTION", 1: "RESULT"}

df = pd.DataFrame(data)
df["label"] = df["label"].map(label2id)

dataset = Dataset.from_pandas(df)


In [ ]:
# Login to Hugging Face

from huggingface_hub import notebook_login

notebook_login()

In [16]:
# Tokenize dataset

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)


Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Train model

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Split dataset
train_test = tokenized_dataset.train_test_split(test_size=0.2)

# Define compute_metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=5,
    report_to="none",
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-17-1941424471.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
5,0.704100
10,0.694600
15,0.643300
20,0.617700
25,0.627900
30,0.545400
35,0.475600
40,0.370400
45,0.336600
50,0.219000


TrainOutput(global_step=250, training_loss=0.12703957223147153, metrics={'train_runtime': 7.1373, 'train_samples_per_second': 140.109, 'train_steps_per_second': 35.027, 'total_flos': 2895654542496.0, 'train_loss': 0.12703957223147153, 'epoch': 10.0})

In [18]:
# Test model

clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

examples = [
    "Buy milk at the supermarket",                                                # ACTION
    "Organize Maria's birthday party",                                            # RESULT
    "Build a new house in the countryside",                                       # RESULT
    "Furnish the new company headquarters",                                       # RESULT
    "Send the monthly report to the CEO",                                         # ACTION
    "Fix the car. I don't know how to fix it",                                    # RESULT
    "Take the car to the repair shop",                                            # ACTION
    "The car is broken and I don't know how to fix it. I have to fix the car",    # RESULT
    "Fix the car at home",                                                        # ACTION
    "I don't have the parts to fix the car. Trying to fix it at home",            # RESULT
    "I have all needed parts to fix the car. I can fix it in 15 minutes",         # ACTION
    "I don't have all needed parts to fix the car. I want to fix it",             # ACTION
    "I have all needed parts to fix the car. I can fix it in 240 minutes",        # RESULT
]


for task in examples:
    prediction = clf(task)[0]
    print(f"Task: {task}")
    print(f" → Prediction: {prediction['label']} (Confidence: {prediction['score']:.2f})\n")


Device set to use cuda:0


Task: Buy milk at the supermarket
 → Prediction: ACTION (Confidence: 0.99)

Task: Organize Maria's birthday party
 → Prediction: RESULT (Confidence: 1.00)

Task: Build a new house in the countryside
 → Prediction: RESULT (Confidence: 1.00)

Task: Furnish the new company headquarters
 → Prediction: RESULT (Confidence: 0.99)

Task: Send the monthly report to the CEO
 → Prediction: ACTION (Confidence: 0.98)

Task: Fix the car. I don't know how to fix it
 → Prediction: RESULT (Confidence: 0.99)

Task: Take the car to the repair shop
 → Prediction: ACTION (Confidence: 1.00)

Task: The car is broken and I don't know how to fix it. I have to fix the car
 → Prediction: RESULT (Confidence: 0.63)

Task: Fix the car at home
 → Prediction: ACTION (Confidence: 1.00)

Task: I don't have the parts to fix the car. Trying to fix it at home
 → Prediction: RESULT (Confidence: 0.95)

Task: I have all needed parts to fix the car. I can fix it in 15 minutes
 → Prediction: ACTION (Confidence: 0.99)

Task: I 

In [19]:
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

examples = [
    "Comprar leche en el supermercado",                    # ACTION
    "Organizar la fiesta de cumpleaños de María",          # RESULT
    "Construir una nueva casa en el campo",                # RESULT
    "Amueblar la nueva sede de la compañía",               # RESULT
    "Enviar el informe mensual al CEO",                    # ACTION
    "Arreglar el coche. No sé cómo hacerlo",               # RESULT
    "Llevar el coche al taller"                            # ACTION
]

for task in examples:
    prediction = clf(task)[0]
    print(f"Task: {task}")
    print(f" → Prediction: {prediction['label']} (Confidence: {prediction['score']:.2f})\n")


Device set to use cuda:0


Task: Comprar leche en el supermercado
 → Prediction: ACTION (Confidence: 0.85)

Task: Organizar la fiesta de cumpleaños de María
 → Prediction: RESULT (Confidence: 0.96)

Task: Construir una nueva casa en el campo
 → Prediction: ACTION (Confidence: 0.61)

Task: Amueblar la nueva sede de la compañía
 → Prediction: ACTION (Confidence: 0.54)

Task: Enviar el informe mensual al CEO
 → Prediction: ACTION (Confidence: 0.96)

Task: Arreglar el coche. No sé cómo hacerlo
 → Prediction: ACTION (Confidence: 0.91)

Task: Llevar el coche al taller
 → Prediction: ACTION (Confidence: 0.87)



In [20]:
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

examples = [
    "Cancel the hotel reservation and notify the travel agency about the change in travel dates",         # ACTION
    "Organize a surprise farewell party for a colleague who is leaving the company next Friday",          # RESULT
    "Fix the sink leak using the wrench and pipe tape I bought from the hardware store",                  # ACTION
    "Plan and coordinate the company’s participation in the annual industry conference in Barcelona",     # RESULT
    "Submit the tax return documents before the deadline to avoid penalties from the tax office",         # ACTION
    "Design, test, and deploy the new landing page for the product marketing campaign",                   # RESULT
    "Pick up the package from the courier office and drop it off at John's desk",                         # ACTION
    "Create a new onboarding process that includes training, documentation, and follow-up support",       # RESULT
    "Call the landlord to report the broken heater and request a maintenance visit this week",            # ACTION
    "Set up a shared folder structure and naming convention for the design team to collaborate",          # RESULT
    "Fill in the reimbursement form and attach all the related receipts before sending it",               # ACTION
    "Develop a monthly reporting system that aggregates data from multiple teams and generates summaries",# RESULT
    "Book the venue for the team offsite and arrange catering and equipment in advance",                  # ACTION
    "Launch a company podcast series featuring internal experts and external guests in each episode",     # RESULT
    "Send a reminder to the finance team about the expense report submission deadline",                   # ACTION
    "Oversee the full migration of the sales database from legacy systems to the new CRM",                # RESULT
    "Write a thank-you email to the client after completing the onboarding session",                      # ACTION
    "Organize the archive room by sorting and labeling all old documents by department and date",         # RESULT
    "Create and deliver a security awareness workshop for all new hires each quarter",                    # RESULT
    "Update your emergency contact information in the HR system before the end of the week",              # ACTION
]

for task in examples:
    prediction = clf(task)[0]
    print(f"Task: {task}")
    print(f" → Prediction: {prediction['label']} (Confidence: {prediction['score']:.2f})\n")

Device set to use cuda:0


Task: Cancel the hotel reservation and notify the travel agency about the change in travel dates
 → Prediction: RESULT (Confidence: 0.94)

Task: Organize a surprise farewell party for a colleague who is leaving the company next Friday
 → Prediction: RESULT (Confidence: 0.98)

Task: Fix the sink leak using the wrench and pipe tape I bought from the hardware store
 → Prediction: ACTION (Confidence: 0.99)

Task: Plan and coordinate the company’s participation in the annual industry conference in Barcelona
 → Prediction: RESULT (Confidence: 1.00)

Task: Submit the tax return documents before the deadline to avoid penalties from the tax office
 → Prediction: RESULT (Confidence: 0.58)

Task: Design, test, and deploy the new landing page for the product marketing campaign
 → Prediction: RESULT (Confidence: 0.99)

Task: Pick up the package from the courier office and drop it off at John's desk
 → Prediction: ACTION (Confidence: 1.00)

Task: Create a new onboarding process that includes trainin